## Import

In [58]:
# 데이터 분석
import pandas as pd
import numpy as np

In [59]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [60]:
# 정규표현식
import re

In [61]:
# 형태소분석 okt
from konlpy.tag import Okt

## 파일 불러오기

In [62]:
df = pd.read_csv('review_data_2t.csv', encoding='utf-8')
df

,review,시설분류
0,?,문화
1,친절해요,문화
2,좋아요 만족스러워요,문화
3,와우~~,문화
4,굿,문화
...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원
141251,친절하십니다,동물병원
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원


## 함수

### 한글만 추출 함수

In [63]:
# 한글만 추출 + 2개이상 공백 1개로 변환
def extract_korean(text):
    # 한글만 추출
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]',' ',text)
    # 공백 2개 이상인 경우 1개로 변환
    text = re.sub(r'\s+',' ',text)
    return text


### 형용사 + 동사 추출 함수

In [64]:
# 형용사 추출하기
def get_adjectives(text):
    okt = Okt() # okt 객체 생성
    okt_pos = okt.pos(text)
    join_text = []
    for text, pos in okt_pos:
        if pos == 'Adjective' or pos == 'Verb':
            join_text.append(text)
    return " ".join(join_text)

## 한글만 추출 (review2 생성)

In [65]:
df['review2'] = df['review'].map(lambda review:extract_korean(review))

In [66]:
df.head(3)

,review,시설분류,review2
0,?,문화,
1,친절해요,문화,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요


## 파생컬럼 생성

In [67]:
# 시간이 오래 걸리는 작업시 시각적으로 진행상황 수치로( %) 보여준다.
from tqdm import tqdm
tqdm.pandas()

In [15]:
df['adj_verb'] = df['review'].progress_map(extract_korean).progress_map(get_adjectives)

100%|█████████████████████████████████████████████████████████████████████████| 141254/141254 [05:26<00:00, 432.68it/s]


In [16]:
df

,review,시설분류,review2,adj_verb
0,?,문화,,
1,친절해요,문화,친절해요,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
3,와우~~,문화,와우,
4,굿,문화,굿,
...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다
141251,친절하십니다,동물병원,친절하십니다,친절하십니다
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요


### 결측치 확인

In [17]:
df.isnull().sum()

review      0
시설분류        0
review2     0
adj_verb    0
dtype: int64

### 데이터의 값이 공백인거 NaN으로 변경

In [18]:
# 데이터값이 공백인거 NaN으로 변경
df_NanTest = df.replace(r'^\s*$', np.nan, regex=True)

In [19]:
df_NanTest

,review,시설분류,review2,adj_verb
0,?,문화,NaN,NaN
1,친절해요,문화,친절해요,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
3,와우~~,문화,와우,NaN
4,굿,문화,굿,NaN
...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다
141251,친절하십니다,동물병원,친절하십니다,친절하십니다
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요


In [20]:
df_NanTest.isnull().sum()

review          9
시설분류            0
review2      4927
adj_verb    30868
dtype: int64

### review2(한글만 남긴 리뷰)의 컬럼 데이터가 NaN이면 제거

In [21]:
df_NanTest.dropna(subset=['review2'], inplace=True)

In [22]:
df_NanTest.isnull().sum()

review          0
시설분류            0
review2         0
adj_verb    25941
dtype: int64

In [26]:
df_NanTest.dropna(subset=['adj_verb'], inplace=True)

In [27]:
df_NanTest.isnull().sum()

review      0
시설분류        0
review2     0
adj_verb    0
dtype: int64

In [29]:
df_NanTest.shape

(110386, 4)

## 저장

In [30]:
df_NanTest.to_csv('data/review_adj_verb.csv', index=False, encoding='utf-8-sig')

## 감성사전으로 라벨링 작업

### 데이터프레임 불러오기

In [68]:
df_review_data = pd.read_csv('data/review_adj_verb.csv', encoding='utf-8')

In [69]:
df_review_data.head(20)

,review,시설분류,review2,adj_verb
0,친절해요,문화,친절해요,친절해요
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
2,좋아요,문화,좋아요,좋아요
3,좋아요,문화,좋아요,좋아요
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요
5,산책하기 좋아요,문화,산책하기 좋아요,하기 좋아요
6,처음 가봤는데 산책로가 잘 되어있네요~\n한적하게 산책하며 힐링하기 좋습니다,문화,처음 가봤는데 산책로가 잘 되어있네요 한적하게 산책하며 힐링하기 좋습니다,가봤는데 잘 되어있네요 한적하게 하며 하기 좋습니다
7,보현사에서 시작~구릉산~갈매구릉산자락길~모다아울렛구리.갈매역까지 걷기\n\n갈매역 ...,문화,보현사에서 시작 구릉산 갈매구릉산자락길 모다아울렛구리 갈매역까지 걷기 갈매역 근처에...,있어 하기 좋음 있어 좋음
8,좋습니다,문화,좋습니다,좋습니다
9,좋아요,문화,좋아요,좋아요


### 감성사전 불러오기

In [70]:
import json

In [71]:
with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    Sentiword_info = json.load(f)

In [72]:
sentiword_dic = pd.DataFrame(Sentiword_info)

In [73]:
sentiword_dic

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


### 정규화, 어간 - 추출 함수

In [74]:
# extract_norm 함수 정의
def extract_norm(words):
    if pd.isna(words):
        return words
    
    okt = Okt()
    norm = []
    for word in words:
        # 정규화
        norm.append(okt.normalize(word))
    return norm

In [75]:
# extract_stem 함수 정의
def extract_stem(words):
    if pd.isna(words):
        return words
     
    okt = Okt()
    stems = []
    for word in words:
        # 어간 추출
        stems.append(okt.morphs(word, stem=True))
    return stems

In [76]:
from tqdm import tqdm

In [77]:
df_review_data['adj_verb_normalized'] = df_review_data['adj_verb'].progress_map(extract_norm)

100%|████████████████████████████████████████████████████████████████████████| 110386/110386 [00:56<00:00, 1960.06it/s]


In [78]:
df_review_data['adj_verb_stemmed'] = df_review_data['adj_verb'].progress_map(extract_stem)

100%|█████████████████████████████████████████████████████████████████████████| 110386/110386 [02:43<00:00, 676.94it/s]


In [79]:
df_review_data

,review,시설분류,review2,adj_verb,adj_verb_normalized,adj_verb_stemmed
0,친절해요,문화,친절해요,친절해요,"[친, 절, 해, 요]","[[친], [절], [해], [요]]"
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,"[좋, 아, 요, , 만, 족, 스, 러, 워, 요]","[[좋다], [아], [요], [], [만], [족], [스], [러], [워], ..."
2,좋아요,문화,좋아요,좋아요,"[좋, 아, 요]","[[좋다], [아], [요]]"
3,좋아요,문화,좋아요,좋아요,"[좋, 아, 요]","[[좋다], [아], [요]]"
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요,"[찾, 아, 간, , 부, 르, 는, , 피, 어, 있, 네, 요]","[[찾다], [아], [간], [], [부], [르], [늘다], [], [피], ..."
...,...,...,...,...,...,...
110381,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...,"[해, 서, , 갔, 어, 요, , 만, 져, , 꼼, 꼼, 히, , 살, ...","[[해], [서다], [], [가다], [어], [요], [], [만], [지다],..."
110382,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다,"[데, 려, 온, , 세, 세, 하, 게, , 봐, 주, 셔, 서, , 드, ...","[[데], [려], [온], [], [세], [세], [하], [게], [], [보..."
110383,친절하십니다,동물병원,친절하십니다,친절하십니다,"[친, 절, 하, 십, 니, 다]","[[친], [절], [하], [십], [니], [다]]"
110384,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요,"[꼼, 꼼, 히, , 봐, 주, 세, 요, , 챙, 겨, 주, 시, 고, , ...","[[꼼], [꼼], [히], [], [보다], [주], [세], [요], [], [..."


### 라벨링 작업

In [ ]:
# df = pd.DataFrame(columns=('adj_verb', 'sentiment'))
# idx = 0 # 다음 리뷰로 넘기기 위한 초기값

# for adj_verb in df_review_data['adj_verb']:  # df_review_data의 adj_verb 열의 각 토큰에 대해 루프
#     sentiment = 0  # 초기 감성값 0으로 설정
#     for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
#         if sentiword_dic.word_root[i] in adj_verb and sentiword_dic.word[i] in adj_verb:  # 리뷰 문장에 감성 단어가 있는지 확인
#             sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
#     df.loc[idx] = [adj_verb, sentiment]  # 리뷰별 극성값을 데이터프레임으로 쌓음
#     idx += 1  # 다음 리뷰 문장으로 넘어감


In [ ]:
# df = pd.DataFrame(columns=('review', '시설분류', 'adj_verb', 'sentiment', 'label'))
# idx = 0 # 다음 리뷰로 넘기기 위한 초기값

# for idx, row in df_review_data.iloc[:20].iterrows():  # 처음 20개의 행에 대해 루프
#     review = row['review']  # 리뷰 가져오기
#     시설분류 = row['시설분류']  # 시설분류 가져오기
#     adj_verb = row['adj_verb']  # adj_verb 가져오기
#     sentiment = 0  # 초기 감성값 0으로 설정
#     for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
#         if sentiword_dic.word_root[i] in adj_verb and sentiword_dic.word[i] in adj_verb:  # 리뷰 문장에 감성 단어가 있는지 확인
#             sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
#     label = 1 if sentiment >= 0 else 0  # sentiment 값에 따라 label 값 할당
#     df.loc[idx] = [review, 시설분류, adj_verb, sentiment, label]  # 리뷰별 극성값과 label을 데이터프레임으로 쌓음


In [50]:
df

,review,시설분류,adj_verb,sentiment,label
0,친절해요,문화,친절해요,0,1
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,4,1
2,좋아요,문화,좋아요,2,1
3,좋아요,문화,좋아요,2,1
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,찾아간 부르는 피어있네요,0,1
5,산책하기 좋아요,문화,하기 좋아요,2,1
6,처음 가봤는데 산책로가 잘 되어있네요~\n한적하게 산책하며 힐링하기 좋습니다,문화,가봤는데 잘 되어있네요 한적하게 하며 하기 좋습니다,0,1
7,보현사에서 시작~구릉산~갈매구릉산자락길~모다아울렛구리.갈매역까지 걷기\n\n갈매역 ...,문화,있어 하기 좋음 있어 좋음,2,1
8,좋습니다,문화,좋습니다,0,1
9,좋아요,문화,좋아요,2,1


In [89]:
# df = pd.DataFrame(columns=('review', '시설분류', 'adj_verb_stemmed', 'sentiment', 'label'))
# idx = 0 # 다음 리뷰로 넘기기 위한 초기값

# for idx, row in df_review_data.iterrows():  # 처음 20개의 행에 대해 루프
#     review = row['review']  # 리뷰 가져오기
#     시설분류 = row['시설분류']  # 시설분류 가져오기
#     adj_verb = row['adj_verb_stemmed']  # adj_verb 가져오기
#     sentiment = 0  # 초기 감성값 0으로 설정
#     for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
#         if sentiword_dic.word_root[i] in adj_verb and sentiword_dic.word[i] in adj_verb:  # 리뷰 문장에 감성 단어가 있는지 확인
#             sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
#     label = 1 if sentiment >= 0 else 0  # sentiment 값에 따라 label 값 할당
#     df.loc[idx] = [review, 시설분류, adj_verb, sentiment, label]  # 리뷰별 극성값과 label을 데이터프레임으로 쌓음


In [ ]:
df = pd.DataFrame(columns=('review', '시설분류', 'adj_verb_stemmed', 'sentiment', 'label'))

for idx, row in tqdm(df_review_data.iterrows(), total=len(df_review_data)):  # tqdm을 사용하여 작업 진행도 표시
    review = row['review']  # 리뷰 가져오기
    시설분류 = row['시설분류']  # 시설분류 가져오기
    adj_verb = row['adj_verb_stemmed']  # adj_verb 가져오기
    sentiment = 0  # 초기 감성값 0으로 설정
    for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
        if sentiword_dic.word_root[i] in adj_verb and sentiword_dic.word[i] in adj_verb:  # 리뷰 문장에 감성 단어가 있는지 확인
            sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
    label = 1 if sentiment >= 0 else 0  # sentiment 값에 따라 label 값 할당
    df.loc[idx] = [review, 시설분류, adj_verb, sentiment, label]  # 리뷰별 극성값과 label을 데이터프레임으로 쌓음

 66%|██████████████████████████████████████████████▊                        | 72749/110386 [2:39:27<1:19:57,  7.84it/s]

In [85]:
df

,review,시설분류,adj_verb,sentiment,label
0,"[[친], [절], [해], [요]]",문화,친절해요,0,1
1,"[[좋다], [아], [요], [], [만], [족], [스], [러], [워], ...",문화,좋아요 만족스러워요,4,1
2,"[[좋다], [아], [요]]",문화,좋아요,2,1
3,"[[좋다], [아], [요]]",문화,좋아요,2,1
4,"[[찾다], [아], [간], [], [부], [르], [늘다], [], [피], ...",문화,찾아간 부르는 피어있네요,0,1
5,"[[하], [기], [], [좋다], [아], [요]]",문화,하기 좋아요,2,1
6,"[[가다], [보다], [늘다], [데], [], [자다], [], [되다], [어...",문화,가봤는데 잘 되어있네요 한적하게 하며 하기 좋습니다,0,1
7,"[[있다], [어], [], [하], [기], [], [좋다], [음], [], [...",문화,있어 하기 좋음 있어 좋음,2,1
8,"[[좋다], [습], [니], [다]]",문화,좋습니다,0,1
9,"[[좋다], [아], [요]]",문화,좋아요,2,1


In [ ]:
df.to_csv('data/test_test.csv', index=False, encoding='utf-8-sig')